# Managing Exceptions with ResultTypes

In this post, we want to briefly review some examples of exception handling, present the [ResultTypes](https://github.com/iamed2/ResultTypes.jl) package as an alternative and finally, show how these mechanisms can cooperate nicely.

## Exceptions in Julia

Julia supports [Exceptions](https://docs.julialang.org/en/v1/manual/control-flow/#Exception-Handling-1) with mechanisms well-known from other languages:

A function can `throw` an exception when facing invalid input or a situation that is in some otherwise unexpected.
The caller can then _handle_ the exception by wrapping the function call in a `try`/`catch` block.
If not handled, the exception will travel upwards on the call stack all the way to the REPL (or even stop the process) and print a stack trace.

Julia offers several built-in exception types, but one can also define custom `Exception` types and implement `Base.showerror` for them to control the printing of error messages.

Finally, there is the `error` function as a shortcut to throw an `ErrorException` with a given message.

In [ ]:
# Example: function uses `error`

In [ ]:
# Example: function throws Exception explicitly

In [ ]:
# Example: function from Base throws Exception

In [ ]:
# Example: custom exception type with custom printing

## ResultTypes

TODO: summarize [ResultTypes](https://github.com/iamed2/ResultTypes.jl), talk about performance benefits, but also hint at others (local error handling, explicit about possible problems).

In [1]:
using ResultTypes

┌ Info: Precompiling ResultTypes [08a2b407-ddc3-586a-afd6-c784ad1fffe2]
└ @ Base loading.jl:1186


In [ ]:
# Example: repeat custom example above

TODO: Discuss scope of `Result`: Can we only apply it _internally_ to our own packages? How to work together with other packages that don't use `Result`?
- Can not avoid that functions we call will throw exceptions (even in Base).
- Can not expect that code that uses our library will deal with `Result`

## Systematic Error-Handling in C++

TODO: Summarize (first part) of Andrei Alexandrescu's talk, with `Expected<T>` type; Compare to `Result`.
[video](https://channel9.msdn.com/Shows/Going+Deep/C-and-Beyond-2012-Andrei-Alexandrescu-Systematic-Error-Handling-in-C)
[slides](https://onedrive.live.com/?cid=F1B8FF18A2AEC5C5&id=F1B8FF18A2AEC5C5%211158&parId=root&o=OneUp)

Focus on slide 27 (_Icing_) with `fromCode`:

```
template <class F>
static Expected fromCode(F fun) {
    try {
        returnExpected(fun());
    } catch(...) {
        return fromException();
    }
}

auto r = Expected<string>::fromCode([] {...});
```

This allows converting capturing exception-throwing code and turn it into `Result` return code!
Even applies to third-party code, can connect the two worlds in both ways!

## Wrapping Callees

In [ ]:
# Example for wrapping called code

# Wrapping for Callers

In [2]:
# Example for throwing from `Result` results